In [1]:
# Here is the complete source code combining all the functions and utilities:

from datetime import datetime, timedelta
from typing import List
import numpy as np
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
from bokeh.models import Range1d,Label
import random
import json
import glob
import os
from scipy.stats import linregress
import math

# 出力設定
output_notebook()

Loading BokehJS ...

In [2]:
def dumpEF(T,F):
    lastT = T[0] ; lastF = F[0]
    for t,f in zip(T[1:],F[1:]):
        dT = round((t - lastT).total_seconds() * 1000,2)
        dF = f - lastF
        print(f'{t} {f}   dT={dT} dF={dF}')
        lastT = t
        lastF = f
    

In [3]:
def normalize_and_filter_exptimes(exptime: List[datetime], fps: float) -> (List[datetime], List[int]):
    """Normalize and filter exptime array to return valid exptimes and their corresponding frame numbers."""
    base_time = exptime[0]
    filtered_frame_numbers = []
    filtered_exptime = []
    ignored_frames = 0  # Count of ignored frames due to abnormal values

    for i, exp_time in enumerate(exptime):
        frame_number = round((exp_time - base_time).total_seconds() * fps,0)
        # Exclude abnormal values
        if i > 0:
            expected_frame_diff = frame_number - filtered_frame_numbers[-1]
            if expected_frame_diff < 0 or expected_frame_diff > 4 + 3 * ignored_frames:
                ignored_frames += 1
                continue
            ignored_frames = 0  # Reset the count as this frame is normal
        filtered_frame_numbers.append(frame_number)
        filtered_exptime.append(exp_time)

    return filtered_exptime, filtered_frame_numbers

In [4]:
def linear_regression_with_fixed_slope(frame_numbers: List[int], exptime: List[datetime], fps: float) -> float:
    """Perform linear regression with a fixed slope of 1/fps and return the y-intercept."""
    times_in_seconds = [(exp_time - exptime[0]).total_seconds() for exp_time in exptime]
    slope = 1 / fps
    intercept = np.mean(times_in_seconds) - slope * np.mean(frame_numbers)
    return intercept

In [5]:
def analyzeExptime(exptime: List[datetime], acctime: List[float], fps: float) -> (List[int], List[datetime], List[float]):
    """Analyzes the exptime[] and returns frame numbers, normalized exptime, and errors.

    Args:
    exptime (List[datetime]): List of datetime objects representing the exposure times.
    acctime (List[float]): List of float values representing the measurement errors in seconds.
    fps (float): The frame rate of the video.

    Returns:
    Tuple[List[int], List[datetime], List[float]]: Frame numbers, normalized exptime, and errors.
    """
    normalized_exptime, frame_numbers = normalize_and_filter_exptimes(exptime, fps)
    y_intercept = linear_regression_with_fixed_slope(frame_numbers, normalized_exptime, fps)
    regression_line = [y_intercept + (1/fps) * frame for frame in frame_numbers]
    errors = [(exp_time - normalized_exptime[0]).total_seconds() - reg_time for exp_time, reg_time in zip(normalized_exptime, regression_line)]

    return frame_numbers, normalized_exptime, errors

In [6]:

def analyzeExptimeImproved(exptime: List[datetime], fps: float) -> (List[int], List[datetime], List[float], float):
    frame_numbers, normalized_exptime, initial_errors = analyzeExptime(exptime, acctime=[], fps=fps)

    # Calculate median and standard deviation of the errors
    median_error = np.median(initial_errors)
    std_error = np.std(initial_errors)

    # Filter out points with error greater than median +/- 2 standard deviations
    error_threshold = 2 * std_error
    filtered_frame_numbers = [fn for fn, err in zip(frame_numbers, initial_errors) if abs(err - median_error) <= error_threshold]
    filtered_exptime = [et for et, err in zip(normalized_exptime, initial_errors) if abs(err - median_error) <= error_threshold]

    # Perform linear regression on the filtered data
    if filtered_frame_numbers and filtered_exptime:
        times_in_seconds = [(exp_time - filtered_exptime[0]).total_seconds() for exp_time in filtered_exptime]
        slope, intercept, _, _, _ = linregress(filtered_frame_numbers, times_in_seconds)
        corrected_fps = 1 / slope if slope != 0 else fps
    else:
        corrected_fps = fps

    # Recalculate errors with the corrected fps
    corrected_errors = [(exp_time - exptime[0]).total_seconds() - fn / corrected_fps for exp_time, fn in zip(normalized_exptime, frame_numbers)]

    return frame_numbers, normalized_exptime, corrected_errors, corrected_fps

# Example usage
# corrected_frame_numbers, corrected_exptime, corrected_errors, corrected_fps = analyzeExptimeImproved(exptime_example, fps)
# Use corrected_fps in drawExptimeError() function




In [7]:
def drawExptimeError(exptime: List[datetime], acctime: List[float], fps: float, figWid=756, figHt=250, title=False, Yrange=False) -> figure:
    frame_numbers, normalized_exptime, errors = analyzeExptime(exptime, acctime, fps)
    p = figure(title=title if title else None, x_axis_label='Frame Number', y_axis_label='Difference (ms)', width=figWid, height=figHt)

    if Yrange:
        p.y_range = Range1d(start=-Yrange, end=Yrange)

    errors_in_ms = [error * 1000 for error in errors]

    for frame_number, error, acc_error in zip(frame_numbers, errors_in_ms, acctime):
        acc_error_range = (acc_error * 1000) / 2
        p.segment(frame_number, error - acc_error_range, frame_number, error + acc_error_range, color="green", line_width=1)

    p.circle(frame_numbers, errors_in_ms, fill_color="red", size=4)

    # Adding a label for the estimated time at Frame 0 with 10 microseconds precision
    frame_0_time = exptime[0] + timedelta(seconds=errors[0])
    label_text = f"Estimated time at Frame 0: {frame_0_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-2]} | fps={fps}"
    label = Label(x=70, y=figHt - 250, x_units='screen', y_units='screen', text=label_text, render_mode='css', 
                  border_line_color='black', border_line_alpha=1.0, background_fill_color='white', background_fill_alpha=1.0)
    p.add_layout(label)

    return p

# Example usage remains the same



In [8]:
def drawExptimeErrorImproved(exptime: List[datetime], acctime: List[float], initial_fps: float, figWid=756, figHt=250, title=False, Yrange=False) -> figure:
    # Use the improved analyzeExptime function to get the corrected fps and other data
    frame_numbers, normalized_exptime, errors, corrected_fps = analyzeExptimeImproved(exptime, initial_fps)

    p = figure(title=title if title else None, x_axis_label='Frame Number', y_axis_label='Difference (ms)', width=figWid, height=figHt)

    if Yrange:
        p.y_range = Range1d(start=-Yrange, end=Yrange)

    errors_in_ms = [error * 1000 for error in errors]  # Convert errors to milliseconds

    # Plotting the error ranges (acctime) in milliseconds
    for frame_number, error, acc_error in zip(frame_numbers, errors_in_ms, acctime):
        acc_error_range = (acc_error * 1000) / 2
        p.segment(frame_number, error - acc_error_range, frame_number, error + acc_error_range, color="lightgreen", line_width=1)

    p.circle(frame_numbers, errors_in_ms, fill_color="red", size=4)

    # Adding a label for the estimated time at Frame 0 with 10 microseconds precision
    if not math.isnan(errors[0]):
        frame_0_time = exptime[0] + timedelta(seconds=errors[0])
    else:
        frame_0_time = exptime[0]
                
    label_text = f"Estimated time at Frame 0: {frame_0_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-2]} | Corrected fps={corrected_fps:.2f}"
    label = Label(x=70, y=figHt - 250, x_units='screen', y_units='screen', text=label_text, render_mode='css', 
                  border_line_color='black', border_line_alpha=1.0, background_fill_color='white', background_fill_alpha=1.0)
    p.add_layout(label)

    return p

# Example usage
# corrected_plot = drawExptimeErrorImproved(exptime_example, acctime_example, fps)
# show(corrected_plot)  # To display the plot (requires 'output_notebook()' for Jupyter Notebooks)



In [52]:
def dumpResult(path,verbose=False,opt_CMCL=False,opt_ACL=False):
    with open(path) as jf:
        RES = json.load(jf)    

    gcb= {}
    lastT = {}
    for FR in RES:
        fn = FR['Frame']
        #print(f'{fn}')
        for BC in FR['GCB']:
            beacon = BC['type']
            bc = BC['devId']

            expTstr = BC['expT']
            if not expTstr:
                continue
            expT = datetime.strptime(expTstr, "%Y-%m-%dT%H:%M:%S.%f")
            acc   = BC['dT']  
            expDur = BC['expDur']
            #print(f'{expT} {acc}')

            RES = BC['result']
            if beacon == 'CL-Beacon':
                ratio = RES['time_cl'][0]
                if opt_ACL:
                    ofs = RES['time'][1][0]     # Original offset
                    ofs_cl = RES['time_acl'][0] # Offset by CL-beacon analitic parser.
                    acc = RES['time_acl'][2]/1000
                    dOfs = ofs_cl - ofs
                    if dOfs >= 0:
                        if dOfs > 0.5:
                            dOfs -= 1.0
                    else:
                        if dOfs < -0.5:
                            dOfs += 1.0
                    expT += + timedelta(milliseconds=dOfs)       
            elif beacon == 'CM-Beacon':
                ratio = RES['time_cm'][0]
                if opt_CMCL:
                    ofs = RES['time'][1][0]          # Original offset(ms)
                    #print(RES['time_cl'])
                    ofs_cl = RES['time_cl'][1][0][1][0] # Offset by CL analysis.(ms)
                    acc = RES['time_cl'][1][0][1][2]/1000  # accuracy ms->s
                    #print(f'\t{round(BC["dT"]*1000,2)} - {round(acc*1000,2)}')
                    #print(f'{ofs}-{ofs_cl}')
                    dOfs = ofs_cl - ofs
                    if dOfs >= 0:
                        if dOfs > 500:
                            dOfs -= 1000
                    else:
                        if dOfs < -500:
                            dOfs += 1000
                    expT += + timedelta(milliseconds=dOfs)
            else:
                ratio = 0

            if verbose:
                print(f'{fn}\t{bc} {expT}({round(acc*1000,2)}ms)  ratio={ratio} ',end='\t')
              
            try:
                gcb[bc]['T'].append(expT)
                gcb[bc]['dT'].append(acc)
                gcb[bc]['ratio'].append(ratio)
                ds  = round((expT - lastT[bc]).total_seconds() * 1000,2)
                if verbose:
                    print(f'{ds}')
                gcb[bc]['dur'].append(ds)
            except KeyError:
                gcb[bc] = {}
                gcb[bc]['T'] = [expT]
                gcb[bc]['dT'] = [acc]
                gcb[bc]['ratio'] = [ratio]
                gcb[bc]['dur'] = []
                gcb[bc]['dTexp'] = BC['result']['dTexp']
                gcb[bc]['expDur'] = expDur
                if verbose:
                    print()
            lastT[bc] = expT            
    for bc in gcb:
        dur = gcb[bc]['dur']
        durAve = sum(dur) / len(dur)
        durVar = sum((x - durAve) ** 2 for x in dur) / len(dur)
        expDur = gcb[bc]['expDur']
        print(f'{bc}\t expDur={round(expDur*1000,2)} , AveFrameDur={round(durAve,3)},fps={round(1000/durAve,2)} , Var={round(durVar,3)}')
    return gcb

In [19]:
def drawResult(file,fps=30,Yrange=False,verbose=False,opt_CMCL=False,opt_ACL=False):
    print(f'drawResult({file})')
    gcb = dumpResult(file,verbose=verbose,opt_CMCL=opt_CMCL,opt_ACL=opt_ACL)
    for bc in gcb:
        timeStamps = gcb[bc]['T']
        timeAccs  = gcb[bc]['dT']
        dTexp = gcb[bc]['dTexp']
        title = f'{bc}'
        if bc == 'CM-Beacon0' and opt_CMCL:
            title += '(as CL)'
        if bc == 'CL-Beacon0' and opt_ACL:
            title += '(ACL)'
        title += f' Time accuracy analysis of \"{file}\", dTexp={round(dTexp*1000,2)}ms'

        fig = drawExptimeErrorImproved(timeStamps,timeAccs,fps,Yrange=Yrange,title=title)
        show(fig)   

In [53]:

#drawResult('result/gcb_0.json',fps=60,Yrange=4)
#drawResult('result/HRC33_lap28.json',fps=240,Yrange=2)
#drawResult('result/HRC33_lap28.json',fps=240,opt_CMCL=True,Yrange=2) # Analyze CM-Beacon as CL-Beacon.

#drawResult('result/I15_NT_SL_H_mid.json',fps=240,Yrange=6)
#drawResult('result/I15_NT_SL_H_mid.json',fps=240,Yrange=6,opt_CMCL=True)
#drawResult('result/I15_NT_SL_V_mid.json',fps=240,Yrange=6)
#drawResult('result/I15_NT_SL_V_mid.json',fps=240,Yrange=6,opt_CMCL=True)

#drawResult('result/I15_NT_H_mid.json',fps=30,Yrange=16,opt_CMCL=False)
#drawResult('result/I15_NT_V_mid.json',fps=30,Yrange=16,opt_CMCL=False)

#drawResult('result/JSB_S_lap2.json',fps=30)
#drawResult('result/JSB_K_lap2.json')
#-----------eval4-------------------------------------------
#drawResult('result/I15_DL_H_30fps.json',fps=30,Yrange=4)
#drawResult('result/I15_DL_H_240fps_A.json',fps=240,Yrange=4)
#drawResult('result/I15_DL_H_240fps_B.json',fps=240,Yrange=4)

#drawResult('result/I15_DL_V_30fps.json',fps=30,Yrange=4,opt_CMCL=False)
#drawResult('result/I15_DL_V_240fps_A.json',fps=240,Yrange=4,opt_CMCL=False)
#drawResult('result/I15_DL_V_240fps_B.json',fps=240,Yrange=4,opt_CMCL=False)

#drawResult('result/GP_DT_H_30fps.json',fps=30,Yrange=4)
#drawResult('result/GP_DT_H_240fps_A.json',fps=240,Yrange=4)
#drawResult('result/GP_DT_V_30fps_A.json',fps=30,Yrange=4)
#drawResult('result/GP_DT_V_30fps_B.json',fps=30,Yrange=4)
#drawResult('result/GP_DT_V_240fps.json',fps=239.80,Yrange=4,opt_CMCL=True)

#drawResult('result/INS_V_Dual.json',fps=30)
#drawResult('result/INS_V_Single.json',fps=30)
#-----------eval5-----------------------------------------------
#drawResult('result/I15_H_30fps_1.json',fps=30)
#drawResult('result/I15_V_30fps_1.json',fps=30)
#drawResult('result/I15_H_240fps_1.json',fps=240,Yrange=4)
#drawResult('result/I15_V_120fps_1.json',fps=120,Yrange=4)
#drawResult('result/I15_V_120fps_5.json',fps=120,Yrange=4)

#drawResult('result/GP_V_30fps_1.json',fps=30,Yrange=4)
#drawResult('result/GP_V_30fps_2.json',fps=30,Yrange=4)
#drawResult('result/GP_V_240fps_1.json',fps=240,Yrange=4)  # 
#drawResult('result/GP_V_240fps_2.json',fps=240,Yrange=4)  # CM-Beaconの直線回帰に失敗している

#drawResult('result/INS_Single_1.json',fps=30,Yrange=4)
drawResult('result/INS_Single_2.json',fps=30,Yrange=4)
drawResult('result/INS_Dual.json',fps=30,Yrange=4)

drawResult(result/INS_Single_2.json)
CL-Beacon0	 expDur=0.42 , AveFrameDur=33.368,fps=29.97 , Var=69.201
CM-Beacon0	 expDur=0.42 , AveFrameDur=33.365,fps=29.97 , Var=66540.44


drawResult(result/INS_Dual.json)
CL-Beacon0	 expDur=0.33 , AveFrameDur=33.366,fps=29.97 , Var=1.645
CM-Beacon0	 expDur=0.33 , AveFrameDur=33.366,fps=29.97 , Var=82.386


In [61]:
def dumpTimeDiff(file,fps=30,Yrange=False,verbose=False,opt_CMCL=False,opt_ACL=False,th=0.008):
    print(f'dumpTimeDiff({file})')
    gcb = dumpResult(file,verbose=verbose,opt_CMCL=opt_CMCL,opt_ACL=opt_ACL)
    TT = []
    for bc in gcb:
        #print(bc)
        TT.append(gcb[bc]['T'])

    lastCl = TT[0][0]
    lastCm = TT[1][0]
    dif = []
    for cl,cm in zip(TT[0][1:],TT[1][1:]):
        dT = (cl-cm).total_seconds()
        if abs(dT) < th:
            dif.append(dT)

    sumD = sum(dif)
    aveD = sumD/len(dif)
    print(f'average difference={round(aveD*1000,2)}')
    return dif

In [59]:
#------------ eval4 -----------------------
# iPhoneを左に９０倒して撮影。　　CLが左上、CMが左下
#dumpTimeDiff('result/I15_DL_H_30fps.json',th=0.008,opt_CMCL=True)    # dT=-0.31ms , 座標差は xxx/1080
#dumpTimeDiff('result/I15_DL_V_30fps.json',th=0.008)                  # dT=-5.5ms , 座標差は 834/1080 => Scan speed:7.12ms
#dumpTimeDiff('result/I15_DL_V_30fps.json',th=0.008,opt_CMCL=True)    # dT=-5.0ms , 座標差は 834/1080 => Scan speed:6.53ms
#dumpTimeDiff('result/I15_DL_V_240fps_A.json',th=0.003)               # dT=-1.4ms, 座標差は 820/1080 => Scan speed:1.84ms
#dumpTimeDiff('result/I15_DL_V_240fps_A.json',th=0.003,opt_CMCL=True)  # dT=-1.3ms, 座標差は 820/1080 => Scan speed:1.71ms

# GoPro
#dumpTimeDiff('result/GP_DT_H_30fps.json',th=0.003)                     # dT=-0.06ms. 　
#dumpTimeDiff('result/GP_DT_V_30fps_A.json',th=0.01)                   # dT=-1.61ms. 　
#dumpTimeDiff('result/GP_DT_V_30fps_A.json',th=0.01,opt_CMCL=True)     # dT=-1.22ms. 　
#dumpTimeDiff('result/GP_DT_H_240fps_A.json',th=0.003)                 # dT=+0.56ms
#dumpTimeDiff('result/GP_DT_H_240fps_A.json',th=0.003,opt_CMCL=True)   # dT=+0.44ms
#dumpTimeDiff('result/GP_DT_V_240fps.json',th=0.003)                   # dT=-1.55ms
#dumpTimeDiff('result/GP_DT_V_240fps.json',th=0.003,opt_CMCL=True)      # dT=-1.40ms

# Insta360 
#dumpTimeDiff('result/INS_V_Dual.json',th=0.02)          # dT=10.8ms,  2カメラの撮影時刻のさは約10.8ms  
#dumpTimeDiff('result/INS_V_Single.json',th=0.02)         # dT=1.65ms,  同一カメラ内でも時間差がある？

#-------------- eval5 ------------------
diff = dumpTimeDiff('result/I15_V_30fps_1.json',th=0.01)                # dT=-5.32ms
#dumpTimeDiff('result/I15_V_30fps_2.json',th=0.01)                 # dT=-5.30ms
#dumpTimeDiff('result/I15_V_120fps_1.json',th=0.01,verbose=True)                 # dT=-3.40ms
#dumpTimeDiff('result/I15_V_120fps_2.json',th=0.01)                 # dT=-3.46ms
#dumpTimeDiff('result/I15_V_120fps_3.json',th=0.01)                 # dT=-3.37ms
#dumpTimeDiff('result/I15_V_120fps_4.json',th=0.01)                 # dT=-3.40ms
#dumpTimeDiff('result/I15_V_120fps_5.json',th=0.01)                 # dT=-3.40ms
#dumpTimeDiff('result/GP_V_30fps_1.json',th=0.01)       # dT=-1.31ms
#dumpTimeDiff('result/GP_V_30fps_2.json',th=0.01)       # dT=-1.60ms
#dumpTimeDiff('result/GP_V_240fps_1.json',th=0.01)       # dT=-1.65ms
#dumpTimeDiff('result/GP_V_240fps_2.json',th=0.01)       # dT=-1.58ms
#dumpTimeDiff('result/INS_Single_1.json',th=0.005)
#dumpTimeDiff('result/INS_Single_2.json',th=0.005)
#dumpTimeDiff('result/INS_Dual.json',th=0.01)           # Insta360 の2個のカメラの時間差はなさそう



dumpTimeDiff(result/I15_V_30fps_1.json)
CL-Beacon0	 expDur=0.43 , AveFrameDur=33.335,fps=30.0 , Var=10.14
CM-Beacon0	 expDur=0.43 , AveFrameDur=33.345,fps=29.99 , Var=64608.674
average error=-5.32


In [ ]:
def draw_multiple_line_plots_with_mode(data_arrays, bins, legends=None, range=None,title = None,legend_pos='top_right'):
    """
    複数のデータ配列とビンの数、範囲を受け取り、Bokehを使用して複数の折れ線グラフを描画する関数。
    各データ配列に対応する凡例（legend）と合計データの最頻値時刻を表示する。

    :param data_arrays: 2次元のデータ配列のリスト（各データ配列はミリ秒単位）
    :param bins: 折れ線グラフのビンの数
    :param legends: 各データ配列の凡例名のリスト
    :param range: 異常値の範囲指定（[min_value, max_value]）
    :return: None
    """
    from bokeh.palettes import Category10
    from bokeh.models import Legend

    # 合計データを準備
    all_data = []

    # Bokehの図を作成
    p = figure(title=title, x_axis_label='Time difference (ms)', y_axis_label='Frequency',
               width=512, height=256)

    # 凡例が提供されていない場合、デフォルトの凡例を作成
    if legends is None:
        legends = [f'Data {i+1}' for i in range(len(data_arrays))]


    # 各データ配列に対して折れ線グラフを描画
    hMax = 0
    for i, data in enumerate(data_arrays):
        # データをミリ秒単位に変換
        data_ms = np.array(data) * 1000

        # 範囲でフィルタリング
        if range is not None:
            range_ms = [x * 1000 for x in range]
            data_ms = data_ms[(data_ms >= range_ms[0]) & (data_ms <= range_ms[1])]

        # 合計データに追加
        all_data.extend(data_ms)
  
        # ヒストグラムのビンを計算
        hist, edges = np.histogram(data_ms, bins=bins, range=range_ms)
        x = (edges[:-1] + edges[1:]) / 2  # 各ビンの中央値
        
        mx = np.max(hist)
        if mx > hMax: hMax = mx

        # 折れ線グラフを描画
        p.line(x, hist, line_width=2, color=Category10[10][i], legend_label=legends[i])

    # 合計データの最頻値時刻を計算
    all_hist, all_edges = np.histogram(all_data, bins=bins, range=range_ms)
    max_index = np.argmax(all_hist)
    mode_time_ms = (all_edges[max_index] + all_edges[max_index + 1]) / 2

    # 最頻値時刻のラベルを追加
    p.add_layout(Label(x=mode_time_ms, y= hMax, text=f'Average difference={mode_time_ms:.2f} ms', text_font_size="10pt", text_baseline="middle"))
    print(f'modeTime={mode_time_ms}, peak_hist={max(all_hist)}')

    # 凡例の位置を設定
    p.legend.location = legend_pos

    # 出力方法を設定（ノートブック上で表示する場合）
    #output_notebook()

    # 折れ線グラフを表示
    show(p)


In [47]:
# 関数を呼び出してヒストグラムを描画

d0 = dumpTimeDiff('result/I15_V_30fps_1.json',th=0.01)                # dT=-5.32ms
d1 = dumpTimeDiff('result/I15_V_30fps_2.json',th=0.01)    
draw_multiple_line_plots_with_mode([d0,d1],legends=['v30-1','v30-2'], range= [-0.01,0.0],bins=100,title="Time difference between two GCBs" )

dumpTimeDiff(result/I15_V_30fps_1.json)
CL-Beacon0	Ave=33.335,fps=30.0 , Var=10.14 , expDur=0.43
CM-Beacon0	Ave=33.345,fps=29.99 , Var=64608.674 , expDur=0.43
average=-5.32
dumpTimeDiff(result/I15_V_30fps_2.json)
CL-Beacon0	Ave=33.335,fps=30.0 , Var=114.093 , expDur=0.4
CM-Beacon0	Ave=39.481,fps=25.33 , Var=134880.255 , expDur=0.4
average=-5.3
modeTime=-4.949999999999999, peak_hist=82


In [62]:
d0 = dumpTimeDiff('result/I15_V_120fps_1.json',th=0.01)                 # dT=-3.40ms
d1 = dumpTimeDiff('result/I15_V_120fps_2.json',th=0.01)                 # dT=-3.46ms
#d2 = dumpTimeDiff('result/I15_V_120fps_3.json',th=0.01)                 # dT=-3.37ms
#d3 = dumpTimeDiff('result/I15_V_120fps_4.json',th=0.01)                 # dT=-3.40ms
d4 = dumpTimeDiff('result/I15_V_120fps_5.json',th=0.01)                 # dT=-3.40ms
d5 = dumpTimeDiff('result/I15_DL_V_240fps_A.json',th=0.01)
d6 = dumpTimeDiff('result/I15_DL_V_240fps_B.json',th=0.01)
#d7 = dumpTimeDiff('result/I15_NT_SL_V_mid.json',th=0.01)
data = [d0,d1,d4,d5,d6]
draw_multiple_line_plots_with_mode(data,
                                   legends=['120fps-1','120fps-2','120fps-3','240fps-1','240fps-2'],
                                   range= [-0.008,0.006],bins=25,
                                   title="Time difference between two GCBs. (iPhone15)",
                                   legend_pos='top_left')

dumpTimeDiff(result/I15_V_120fps_1.json)
CL-Beacon0	 expDur=0.35 , AveFrameDur=8.339,fps=119.91 , Var=74.402
CM-Beacon0	 expDur=0.35 , AveFrameDur=8.206,fps=121.87 , Var=2769.948
average difference=-3.4
dumpTimeDiff(result/I15_V_120fps_2.json)
CL-Beacon0	 expDur=0.34 , AveFrameDur=8.339,fps=119.92 , Var=74.459
CM-Beacon0	 expDur=0.34 , AveFrameDur=8.339,fps=119.91 , Var=9616.337
average difference=-3.46
dumpTimeDiff(result/I15_V_120fps_5.json)
CL-Beacon0	 expDur=0.43 , AveFrameDur=8.338,fps=119.93 , Var=1.572
CM-Beacon0	 expDur=0.43 , AveFrameDur=9.408,fps=106.29 , Var=1711.034
average difference=-3.4
dumpTimeDiff(result/I15_DL_V_240fps_A.json)
CL-Beacon0	 expDur=0.77 , AveFrameDur=7.464,fps=133.97 , Var=10481.257
CM-Beacon0	 expDur=0.77 , AveFrameDur=5.938,fps=168.42 , Var=21180.648
average difference=-1.73
dumpTimeDiff(result/I15_DL_V_240fps_B.json)
CL-Beacon0	 expDur=0.64 , AveFrameDur=33.271,fps=30.06 , Var=562.692
CM-Beacon0	 expDur=0.64 , AveFrameDur=33.313,fps=30.02 , Var=49095.

In [56]:
d0 = dumpTimeDiff('result/I15_V_30fps_1.json',th=0.01)
d1 = dumpTimeDiff('result/I15_V_30fps_2.json',th=0.01)
d2 = dumpTimeDiff('result/I15_V_30fps_3.json',th=0.01)
d3 = dumpTimeDiff('result/I15_DL_V_30fps.json',th=0.01)
data = [d0,d1,d2,d3]
draw_multiple_line_plots_with_mode(data,
                                   legends=['30fps-1','30fps-2','30fps-3','30fps-4'],
                                   range= [-0.008,0.006],bins=28,
                                   title="Time difference between diagonal GCBs. (iPhone15)",
                                   legend_pos='top_right')

dumpTimeDiff(result/I15_V_30fps_1.json)
CL-Beacon0	 expDur=0.43 , AveFrameDur=33.335,fps=30.0 , Var=10.14
CM-Beacon0	 expDur=0.43 , AveFrameDur=33.345,fps=29.99 , Var=64608.674
average=-5.32
dumpTimeDiff(result/I15_V_30fps_2.json)
CL-Beacon0	 expDur=0.4 , AveFrameDur=33.335,fps=30.0 , Var=114.093
CM-Beacon0	 expDur=0.4 , AveFrameDur=39.481,fps=25.33 , Var=134880.255
average=-5.3
dumpTimeDiff(result/I15_V_30fps_3.json)
CL-Beacon0	 expDur=0.57 , AveFrameDur=33.336,fps=30.0 , Var=0.307
CM-Beacon0	 expDur=0.57 , AveFrameDur=33.137,fps=30.18 , Var=32628.834
average=-5.18
dumpTimeDiff(result/I15_DL_V_30fps.json)
CL-Beacon0	 expDur=0.63 , AveFrameDur=33.335,fps=30.0 , Var=104.799
CM-Beacon0	 expDur=0.63 , AveFrameDur=37.486,fps=26.68 , Var=167438.739
average=-5.69
modeTime=-5.25, peak_hist=158


In [57]:
#
#  iPhone15 , Holizontal position time difference analysis.
#
d0 = dumpTimeDiff('result/I15_H_30fps_1.json',th=0.01)   
d1 = dumpTimeDiff('result/I15_H_30fps_2.json',th=0.01) 
d2 = dumpTimeDiff('result/I15_DL_H_30fps.json',th=0.01)
d3 = dumpTimeDiff('result/I15_NT_H_mid.json',th=0.01)

d4 = dumpTimeDiff('result/I15_H_240fps_1.json',th=0.01)
#d4 = dumpTimeDiff('result/I15_H_240fps_2.json',th=0.01)
d5 = dumpTimeDiff('result/I15_DL_H_240fps_A.json',th=0.01)
d6 = dumpTimeDiff('result/I15_DL_H_240fps_B.json',th=0.01)

data = [d0,d1,d2,d3,d4,d5,d6]
draw_multiple_line_plots_with_mode(data,
                                   legends=['30fps-1','30fps-2','30fps-3','30fps-4','240fps-1','240fps-2','240fps-3'],
                                   range= [-0.008,0.006],bins=28,
                                   title="Time difference between holizontal GCBs. (iPhone15)",
                                   legend_pos = 'top_left')

dumpTimeDiff(result/I15_H_30fps_1.json)
CL-Beacon0	 expDur=0.38 , AveFrameDur=33.336,fps=30.0 , Var=4.411
CM-Beacon0	 expDur=0.38 , AveFrameDur=33.338,fps=30.0 , Var=37877.717
average=-0.03
dumpTimeDiff(result/I15_H_30fps_2.json)
CL-Beacon0	 expDur=0.38 , AveFrameDur=33.336,fps=30.0 , Var=12.78
CM-Beacon0	 expDur=0.38 , AveFrameDur=33.298,fps=30.03 , Var=10.477
average=-0.23
dumpTimeDiff(result/I15_DL_H_30fps.json)
CL-Beacon0	 expDur=0.68 , AveFrameDur=33.335,fps=30.0 , Var=6.176
CM-Beacon0	 expDur=0.68 , AveFrameDur=33.337,fps=30.0 , Var=58251.767
average=-0.58
dumpTimeDiff(result/I15_NT_H_mid.json)
CL-Beacon0	 expDur=12.53 , AveFrameDur=27.312,fps=36.61 , Var=55241.127
CM-Beacon0	 expDur=12.53 , AveFrameDur=33.355,fps=29.98 , Var=7.351
average=1.75
dumpTimeDiff(result/I15_H_240fps_1.json)
CL-Beacon0	 expDur=0.34 , AveFrameDur=5.907,fps=169.28 , Var=11.742
CM-Beacon0	 expDur=0.34 , AveFrameDur=5.907,fps=169.28 , Var=3121.445
average=-0.1
dumpTimeDiff(result/I15_DL_H_240fps_A.json)
CL-

In [63]:
d0 = dumpTimeDiff('result/GP_V_30fps_1.json',th=0.01)       # dT=-1.31ms
d1 = dumpTimeDiff('result/GP_V_30fps_2.json',th=0.01)       # dT=-1.60ms
d2 = dumpTimeDiff('result/GP_V_240fps_1.json',th=0.01)
d3 = dumpTimeDiff('result/GP_V_240fps_2.json',th=0.01)
data = [d0,d1,d2,d3]
draw_multiple_line_plots_with_mode(data,
                                   legends=['30fps-1','30fps-2','240fps-1','240fps-2'],
                                   range= [-0.008,0.006],bins=28,
                                   title="Time difference between diagonal GCBs. (GoPro9)",
                                   legend_pos = 'top_left')

dumpTimeDiff(result/GP_V_30fps_1.json)
CL-Beacon0	 expDur=0.67 , AveFrameDur=33.364,fps=29.97 , Var=14.643
CM-Beacon0	 expDur=0.67 , AveFrameDur=33.364,fps=29.97 , Var=15478.455
average difference=-1.31
dumpTimeDiff(result/GP_V_30fps_2.json)
CL-Beacon0	 expDur=0.71 , AveFrameDur=33.364,fps=29.97 , Var=40.84
CM-Beacon0	 expDur=0.71 , AveFrameDur=34.137,fps=29.29 , Var=12169.374
average difference=-1.6
dumpTimeDiff(result/GP_V_240fps_1.json)
CL-Beacon0	 expDur=0.7 , AveFrameDur=4.169,fps=239.86 , Var=10.752
CM-Beacon0	 expDur=0.7 , AveFrameDur=4.171,fps=239.74 , Var=13.562
average difference=-1.65
dumpTimeDiff(result/GP_V_240fps_2.json)
CL-Beacon0	 expDur=0.68 , AveFrameDur=4.173,fps=239.62 , Var=40.035
CM-Beacon0	 expDur=0.68 , AveFrameDur=4.171,fps=239.74 , Var=83.055
average difference=-1.58
modeTime=-1.75, peak_hist=488


In [64]:
d0 = dumpTimeDiff('result/GP_DT_H_30fps.json',th=0.01) 
d1 = dumpTimeDiff('result/GP_DT_H_240fps_A.json',th=0.01) 
d2 = dumpTimeDiff('result/GPR_NT_H_mid.json',th=0.01) 
data = [d0,d1,d2]
draw_multiple_line_plots_with_mode(data,
                                   legends=['30fps-1','240fps','30fps-2'],
                                   range= [-0.008,0.006],bins=28,
                                   title="Time difference between diagonal GCBs. (GoPro9)",
                                   legend_pos = 'top_left')

dumpTimeDiff(result/GP_DT_H_30fps.json)
CL-Beacon0	 expDur=1.12 , AveFrameDur=33.364,fps=29.97 , Var=0.82
CM-Beacon0	 expDur=1.12 , AveFrameDur=33.36,fps=29.98 , Var=3725.893
average difference=0.08
dumpTimeDiff(result/GP_DT_H_240fps_A.json)
CL-Beacon0	 expDur=2.2 , AveFrameDur=4.175,fps=239.5 , Var=3.393
CM-Beacon0	 expDur=2.2 , AveFrameDur=3.641,fps=274.65 , Var=729.903
average difference=0.56
dumpTimeDiff(result/GPR_NT_H_mid.json)
CL-Beacon0	 expDur=4.28 , AveFrameDur=33.103,fps=30.21 , Var=24.625
CM-Beacon0	 expDur=4.28 , AveFrameDur=-0.47,fps=-2125.98 , Var=224032.267
average difference=1.34
modeTime=0.25, peak_hist=353
